マルチ学習のススメ
--------------------------

単一GPUだけで計算をするのは限界がある．そこで2つ以上のGPUで学習を行う方法をここでは紹介する．

**大きく分けてDataParallelによるマルチ学習とDistributedDataParallelによるマルチ学習がある**．

前者は**手軽**に試せる．

後者は手間はかかるが**訓練速度がさらに速くなる**のでおすすめ．

今回はDistributedDataParallellについて詳しく見ていく

DataParallelとDistributedDataParallelの違い
---------------
気になる人だけ見ればおk

DataParallelはシングルプロセスかつマルチスレッドのみで機能するがDistributedDataParallelはさらにマルチプロセスでシングルマシン訓練、マルチマシン訓練でも機能する．
また，通常、DataParallel は、スレッド間のGILの競合、イテレーション毎に複製するモデル、そして入力の分割と出力の収集によって発生するオーバーヘッドが原因となり、単一のマシン上であってもDistributedDataParallel よりも遅くなる．

モデルが大きすぎて単一のGPUに収まらない場合は、モデル並列を利用してモデルを複数のGPUに分割する必要があるが，DistributedDataParallel は、モデル並列と共に動作できる一方で DataParallel はモデル並列と共に使うことはできない．

などが主な理由らしい

In [ ]:
import os
import sys
import tempfile
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.optim as optim
import torch.multiprocessing as mp

from torch.nn.parallel import DistributedDataParallel as DDP


def setup(rank, world_size):
    if sys.platform == 'win32':
        # Windowsプラットフォーム上では、Distribuedパッケージは
        # Glooバックエンドの集合通信のみサポートしています。
        # init_process_group内のinit_method パラメーターをローカルのファイルに設定してください。
        # 例 init_method="file:///f:/libtmp/some_file"
        init_method="file:///{your local file path}"

        # プロセスグループの初期化
        dist.init_process_group(
            "gloo",
            init_method=init_method,
            rank=rank,
            world_size=world_size
        )
    else:
        os.environ['MASTER_ADDR'] = 'localhost'
        os.environ['MASTER_PORT'] = '12355'

        # プロセスグループの初期化
        dist.init_process_group("gloo", rank=rank, world_size=world_size)

def cleanup():
    dist.destroy_process_group()

簡単なモジュールを作ってDDPでラップする．

In [ ]:
class ToyModel(nn.Module):
    def __init__(self):
        super(ToyModel, self).__init__()
        self.net1 = nn.Linear(10, 10)
        self.relu = nn.ReLU()
        self.net2 = nn.Linear(10, 5)

    def forward(self, x):
        return self.net2(self.relu(self.net1(x)))


def demo_basic(rank, world_size):
    print(f"Running basic DDP example on rank {rank}.")
    setup(rank, world_size)

    # モデルを作成し、ランクidと共にGPUに移動
    model = ToyModel().to(rank)
    ddp_model = DDP(model, device_ids=[rank])

    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(ddp_model.parameters(), lr=0.001)

    optimizer.zero_grad()
    outputs = ddp_model(torch.randn(20, 10))
    labels = torch.randn(20, 5).to(rank)
    loss_fn(outputs, labels).backward()
    optimizer.step()

    cleanup()


def run_demo(demo_fn, world_size):
    mp.spawn(demo_fn,
             args=(world_size,),
             nprocs=world_size,
             join=True)

DDPを行う場合でも並列処理でない場合とほとんど同様の実装を使える．

今回はかなり簡易的な実装だがもう少し発展的な実装の場合必要になる処理があるので下に記載．

## 処理速度の違い

DDPでは、コンストラクター、フォワードパス、そしてバックワードパスが分散処理の同期ポイントになる．

そして、異なるプロセスは同じ数の同期処理を起動し、同じ順序でこれらの同期ポイントに到達し、ほぼ同時に各同期ポイントに入ることが期待されている．

このようにしないと、速く行われるプロセスが早く到着し、出遅れたプロセスを待ってタイムアウトしてしまう可能性がある。

したがって、ユーザーにはプロセス間でワークロードの分散を均等にする必要がある。

しかし、処理速度の歪み、バラつきは、例えば、ネットワークの遅延、リソースの競合、または予測不能なワークロードの急増によって不可避的に発生する。

このような状況でのタイムアウトを避けるには、init_process_group を呼び出す際に、十分な timeout 値を与えておく。

In [ ]:
def demo_model_parallel(rank, world_size):
    print(f"Running DDP with model parallel example on rank {rank}.")
    setup(rank, world_size)

    # このプロセスで使用するmp_modelとデバイスをセットアップ
    dev0 = rank * 2
    dev1 = rank * 2 + 1
    mp_model = ToyMpModel(dev0, dev1)
    ddp_mp_model = DDP(mp_model)

    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(ddp_mp_model.parameters(), lr=0.001)

    optimizer.zero_grad()
    # 出力は dev1 に行われる。
    outputs = ddp_mp_model(torch.randn(20, 10))
    labels = torch.randn(20, 5).to(dev1)
    loss_fn(outputs, labels).backward()
    optimizer.step()

    cleanup()


if __name__ == "__main__":
    n_gpus = torch.cuda.device_count()
    if n_gpus < 8:
        print(f"Requires at least 8 GPUs to run, but got {n_gpus}.")
    else:
        run_demo(demo_basic, 8)
        run_demo(demo_checkpoint, 8)
        run_demo(demo_model_parallel, 4)

Requires at least 8 GPUs to run, but got 1.


以前のGoogle Colaoratoryなら上手くいったコードだが，改悪によって利用できなくなってしまった．

Googleドキュメントに別の形でDDPについて解説しておくのでそちらを見るのが吉．